In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import smartcitizen_connector
import scdata as sc
import time
now = time.localtime()

In [ ]:
df = smartcitizen_connector.search_by_query(endpoint='devices',
                                            search_items=[{
                                                'key': 'postprocessing_id', 
                                                'value': 'not_null',
                                                'full': True
                                            }])

In [ ]:
# Find by MAC addresses
devices = []
mac_addresses = ['CF65', 'C9F0', '7D71', 'B5E9', 'C8C7', 'D3CD']

In [ ]:
for item in df['postprocessing']:
    name = df.loc[item['device_id'], 'name']
    # print (item['device_id'], df.loc[item['device_id'], 'name'], item['meta'])
    if item['meta'] is None: continue
    if 'twinair' not in item['meta'].lower(): continue
    if '#' in name:
        ind = name.index('#') + 1
        if name[ind:] in mac_addresses: 
            print (f"Found: {name[ind:]} - {item['device_id']}")
            devices.append(item['device_id'])
        else:
            d = smartcitizen_connector.SCDevice(item['device_id'])
            mac = d.last_status_message.mac.replace(':','')[-4:]
            if mac in mac_addresses: 
                print (f"Digging deeper, found: {name[ind:]} - {item['device_id']}")
                devices.append(item['device_id'])

In [ ]:
devices

In [ ]:
macs = {}
for device in devices:
    a = smartcitizen_connector.SCDevice(device)
    macs[device] = a.last_status_message

In [ ]:
macs

In [ ]:
devices_api = [sc.Device(blueprint='sc_air', params=sc.APIParams(id=id)) for id in devices]

In [ ]:
devices_csv = [sc.Device(blueprint='sc_air',
                        source={'type':'csv', 
                                'handler': 'CSVHandler',
                                'module': 'scdata.io.device_file'},
                        params=sc.CSVParams(id=0, 
                                            path=f'csv/CO2 Chamber Testing SCKs_7545Test 001_raw.csv',
                                            header_skip=[1],
                                            separator='\t',
                                            timezone='Europe/Madrid')
                      )]

In [ ]:
test = sc.Test(name=f'CO2_CHAMBER_TEST',
            devices=devices_api + devices_csv,
            force_recreate=True
)
test.options.cache=True

In [ ]:
await test.load()

In [ ]:
import pandas as pd

In [ ]:
test.get_device(0).data.index = test.get_device(0).data.index - pd.Timedelta(hours=1) 

In [ ]:
traces = {
            "1": {"devices": [17265, 17295, 17313, 17318, 17319, 17306],
                  "channel": "SCD30_CO2",
                  "subplot": 1},
            "2": {"devices": [0],
                  "channel": "CO2",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'CO2',
                            'padding-bottom': 600,
                            'ylabel': {1: 'CO2 (ppm)'}
                           }, 
              options = {'frequency': '1Min', 
                         'min_date':'2024-05-22 18:50:00', 
                         'max_date':'2024-05-24 16:15:00'
                         })